In [12]:
# セル1: 共通ベースクラス（BaseScraper）

"""
BaseScraper - スクレイピングの基本機能を提供する基底クラス

このモジュールはスクレイピングの共通機能を持つ基底クラスを提供します。
設定ファイルの読み込み、ブラウザの初期化、ログ機能などの共通機能を集約しています。
"""

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import os
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import logging
from dotenv import load_dotenv
from pathlib import Path
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
import yaml
import json


class BaseScraper:
    """
    スクレイピングの基本機能を提供する基底クラス
    
    このクラスはスクレイピングに必要な以下の共通機能を提供します:
    - 設定ファイルの読み込み
    - ブラウザの初期化と設定
    - ログ機能
    - CSVファイル操作
    """

    def _load_config(self, config_path=None):
        """
        設定ファイルを読み込みます
        
        Args:
            config_path (str): 設定ファイルのパス
        
        Returns:
            dict: 設定データ
        """
        # デフォルトのパス
        if config_path is None:
            config_path = os.path.join(self.root_dir, 'config', 'settings.yaml')
        
        print(f"設定ファイルパス: {config_path}")
        
        try:
            if not os.path.exists(config_path):
                raise FileNotFoundError(f"設定ファイルが見つかりません: {config_path}")
                
            # YAMLファイルを読み込む
            with open(config_path, 'r', encoding='utf-8') as f:
                config = yaml.safe_load(f)
            
            # 環境変数から認証情報を取得して設定ファイルにマージ
            self._merge_env_variables(config)
            
            print("設定ファイルを正常に読み込みました")
            return config
        except Exception as e:
            print(f"設定ファイルの読み込みエラー: {str(e)}")
            # エラーを上位に伝播させる
            raise

    def load_config(config_path=None):
        """
        設定ファイルを読み込み、環境変数で値を置き換えます
        
        Args:
            config_path (str): 設定ファイルのパス
        
        Returns:
            dict: 設定データ
        """
        # .envファイルを読み込み
        load_dotenv()
        
        # デフォルトのパス
        if config_path is None:
            # プロジェクトルートを特定
            current_dir = os.getcwd()
            project_root = current_dir
            
            # notebooksディレクトリにいる場合はルートに戻る
            if os.path.basename(current_dir) == 'notebooks':
                project_root = os.path.dirname(current_dir)
                
            config_path = os.path.join(project_root, "config", "settings.yaml")
        
        print(f"設定ファイルパス: {config_path}")
        
        try:
            if not os.path.exists(config_path):
                raise FileNotFoundError(f"設定ファイルが見つかりません: {config_path}")
                
            with open(config_path, 'r', encoding='utf-8') as f:
                # YAMLファイルを読み込む
                yaml_content = f.read()
                
                # 環境変数プレースホルダーを置換
                pattern = r'\$\{([A-Za-z0-9_]+)\}'
                
                def replace_env_var(match):
                    env_var = match.group(1)
                    return os.environ.get(env_var, f"${{{env_var}}}")
                
                processed_yaml = re.sub(pattern, replace_env_var, yaml_content)
                
                # 処理済みYAMLを解析
                config = yaml.safe_load(processed_yaml)
            
            print("設定ファイルを正常に読み込みました")
            return config
        except Exception as e:
            print(f"設定ファイルの読み込みエラー: {str(e)}")
            # エラーを上位に伝播させる
            raise

    def _merge_env_variables(self, config):
        """環境変数から認証情報を取得し、設定ファイルにマージする"""
        # このメソッドは子クラスでオーバーライドして実装します
        pass

    def setup_logging(self):
        """ログ機能のセットアップ"""
        # すでに存在するハンドラを削除（重複を防ぐため）
        for handler in logging.root.handlers[:]:
            logging.root.removeHandler(handler)
        
        # ログファイルパスの設定
        log_filename = f'{self.site_name}_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'
        log_file = os.path.join(self.log_dir, log_filename)
        
        # 基本設定
        logging.basicConfig(
            filename=log_file,
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            encoding='utf-8'
        )
        
        # コンソールにもログを出力
        console = logging.StreamHandler()
        console.setLevel(logging.INFO)
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        console.setFormatter(formatter)
        logging.getLogger('').addHandler(console)
        
        # ログファイルの場所を明示的に表示
        print(f"ログファイル出力先: {log_file}")
        logging.info(f"ログ機能の初期化が完了しました: {log_file}")

    def _find_project_root(self):
        """
        プロジェクトのルートディレクトリを検出する
        """
        # 現在のファイルの絶対パスを取得
        current_dir = os.path.abspath(os.getcwd())
        
        # 親ディレクトリを探索
        path = Path(current_dir)
        while True:
            # .gitディレクトリがあればそれをルートとみなす
            if (path / '.git').exists():
                return str(path)
            
            # プロジェクトのルートを示す他のファイル/ディレクトリの存在チェック
            if (path / 'setup.py').exists() or (path / 'README.md').exists():
                return str(path)
            
            # これ以上上の階層がない場合は現在のディレクトリを返す
            if path.parent == path:
                return str(path)
            
            # 親ディレクトリへ
            path = path.parent

    def _setup_browser(self):
        """
        Seleniumのブラウザを設定します
        
        ChromeOptionsの設定とWebDriverの起動を行います
        """
        # Chromeオプション設定
        chrome_options = Options()
        if self.headless_mode:
            chrome_options.add_argument("--headless")  # ヘッドレスモード（画面表示なし）
        chrome_options.add_argument("--disable-gpu")  # GPU無効化（ヘッドレス時に推奨）
        chrome_options.add_argument("--window-size=1920x1080")  # ウィンドウサイズ設定
        
        try:
            # ドライバーパスが指定されている場合は直接使用
            if hasattr(self, 'driver_path') and self.driver_path:
                self.browser = webdriver.Chrome(service=Service(self.driver_path), options=chrome_options)
            # 指定がなければWebDriverManagerを使用
            else:
                self.browser = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)
            
            self.wait = WebDriverWait(self.browser, 10)  # 要素が見つかるまで最大10秒待機
            print("ブラウザの初期化に成功しました")
        except Exception as e:
            print(f"ブラウザの初期化に失敗しました: {str(e)}")
            raise

    def _setup_session(self):
        """
        requestsのセッションを設定します
        
        Seleniumのブラウザから取得したCookieをrequestsセッションに設定します
        これによりログイン状態を維持したままBeautifulSoupでページ取得できます
        """
        # ブラウザのCookieを取得
        cookies = self.browser.get_cookies()
        
        # requestsセッションを作成
        self.session = requests.Session()
        
        # Seleniumから取得したCookieをrequestsセッションに追加
        for cookie in cookies:
            self.session.cookies.set(cookie['name'], cookie['value'])

    def prepare_csv(self):
        """
        CSVファイルを初期化します
        
        既存のファイルがある場合は削除し、新しいヘッダー付きのCSVを作成します
        """
        # 既存ファイルがあれば削除
        if os.path.exists(self.csv_filename):
            os.remove(self.csv_filename)
        
        # カラム構成でCSVを初期化
        df = pd.DataFrame(columns=self.columns)
        df.to_csv(self.csv_filename, index=False, encoding="utf-8-sig")
        
        print(f"CSVファイル {self.csv_filename} を初期化しました")

    def save_to_csv(self, data):
        """
        データをCSVファイルに保存します
        
        Args:
            data (list): 保存するデータ（リストのリスト形式）
            
        Returns:
            bool: 保存成功時はTrue、失敗時はFalse
        """
        try:
            if data:
                # データフレームに変換
                df = pd.DataFrame(data, columns=self.columns)
                
                # CSVに追記（ヘッダーなし）
                df.to_csv(self.csv_filename, mode="a", index=False, header=False, encoding="utf-8-sig")
                return True
            else:
                print("保存するデータがありません")
                return False
        except Exception as e:
            print(f"CSV保存エラー: {str(e)}")
            return False

    def __init__(self, site_name, config_path=None, headless_mode=False):
        """
        BaseScraperの初期化
        
        Args:
            site_name (str): スクレイピングするサイト名（ログファイル名などに使用）
            config_path (str): 設定ファイルのパス（指定しない場合はデフォルト値を使用）
            headless_mode (bool): ブラウザを画面に表示せずに実行する場合はTrue
        """
        # サイト名の設定
        self.site_name = site_name
        
        # プロジェクトルートディレクトリの検出
        self.root_dir = self._find_project_root()
        
        # 環境変数の読み込み
        load_dotenv(os.path.join(self.root_dir, '.env'))
        
        # ディレクトリパスの設定
        self.data_dir = os.path.join(self.root_dir, 'data')
        self.log_dir = os.path.join(self.root_dir, 'logs')
        
        # ディレクトリが存在しない場合は作成
        os.makedirs(self.data_dir, exist_ok=True)
        os.makedirs(self.log_dir, exist_ok=True)
        
        # 設定ファイルの読み込み
        self.config = self._load_config(config_path)
        
        # 基本設定
        self.headless_mode = headless_mode
        self.browser = None
        self.wait = None
        self.session = None
        
        # 実行時間計測用
        self.start_time = None
        self.end_time = None

        # ログ設定を初期化時に行う
        self.setup_logging()

    def close(self):
        """
        ブラウザを閉じてリソースを解放します
        """
        if self.browser:
            self.browser.quit()
            print("ブラウザを閉じました")

In [14]:
# セル2: SUPERDElibARY個別クラス（NetseaScraper）

"""
NetseaScraper - ネッシーから商品情報を取得するスクレイピングモジュール

このモジュールはネッシーのウェブサイトから商品情報を取得するための
クラスを提供します。BaseScraper を継承し、NETSEA固有の機能を実装しています。
"""

class SudeliScraper(BaseScraper):
    """
    スーパーデリバリーのWebサイトから商品情報をスクレイピングするクラス
    
    商品一覧ページから直接情報を取得するため、個別の商品ページにアクセスする必要がありません。
    """
    
    def _merge_env_variables(self, config):
        """環境変数から認証情報を取得し、設定ファイルにマージする"""
        # スーパーデリバリーの認証情報
        username = os.getenv('SUDELI_USERNAME')
        password = os.getenv('SUDELI_PASSWORD')
        
        if username and password:
            if 'scrapers' not in config:
                config['scrapers'] = {}
            if 'sudeli' not in config['scrapers']:
                config['scrapers']['sudeli'] = {}
            if 'login' not in config['scrapers']['sudeli']:
                config['scrapers']['sudeli']['login'] = {}
            
            config['scrapers']['sudeli']['login']['username'] = username
            config['scrapers']['sudeli']['login']['password'] = password
            print("スーパーデリバリーのログイン情報を環境変数から設定しました")
        else:
            print("警告: 環境変数からスーパーデリバリーのログイン情報を取得できませんでした")
            
        # デフォルトの出力設定（なければ設定）
        if 'output' not in config['scrapers']['sudeli']:
            config['scrapers']['sudeli']['output'] = {
                'csv_filename': 'sudeli_scraping.csv',
                'log_dir': 'logs'
            }
    
    def __init__(self, config_path=None, headless_mode=False):
        """
        SudeliScraperの初期化
        
        Args:
            config_path (str): 設定ファイルのパス（指定しない場合はデフォルト値を使用）
            headless_mode (bool): ブラウザを画面に表示せずに実行する場合はTrue
        """
        # 親クラス(BaseScraper)の初期化
        super().__init__('sudeli', config_path, headless_mode)
        
        # SUDELI固有の設定
        self.sudeli_config = self.config['scrapers']['sudeli']
        self.base_url = "https://www.superdelivery.com"
        
        # 出力設定（設定ファイルから読み込み）
        output_config = self.sudeli_config.get('output', {})
        csv_filename = output_config.get('csv_filename', "sudeli_scraping.csv")
        
        # CSVのフルパスを設定
        self.csv_filename = os.path.join(self.data_dir, csv_filename)
        print(f"CSVファイル出力先: {self.csv_filename}")
        
        # カラム設定（NETSEA固有）
        self.columns = ["卸業者名", "商品名", "JANコード", "価格", "セット数"]
        
        # ブラウザ設定
        self._setup_browser()
    
        
    
    def login(self, username=None, password=None):
        """
        スーパーデリバリーにログインします
        
        Args:
            username (str): スーパーデリバリーのログインユーザー名（指定なしの場合は設定ファイルから読み込み）
            password (str): スーパーデリバリーのパスワード（指定なしの場合は設定ファイルから読み込み）
            
        Returns:
            bool: ログイン成功時はTrue、失敗時はFalse
        """
        # ユーザー名とパスワードが指定されていない場合は設定ファイルから読み込む
        if username is None or password is None:
            username = self.sudeli_config['login']['username']
            password = self.sudeli_config['login']['password']
            
        try:
            # ログインページにアクセス
            self.browser.get(f"{self.base_url}/p/do/clickMemberLogin")
            
            # ログインフォームに入力
            self.wait.until(EC.presence_of_element_located((By.NAME, "identification"))).send_keys(username)
            self.browser.find_element(By.NAME, "password").send_keys(password)
            
            # ログインボタンのクリック - input[type="submit"]を選択
            self.browser.find_element(By.CSS_SELECTOR, "input[type='submit'][value='ログイン']").click()
            
            # ログイン成功の確認（ログイン後ページの特定要素が表示されるか）
            self.wait.until(EC.presence_of_element_located((By.ID, "loading-contents")))
            
            # ログイン後、Cookieを取得してrequestsセッションを準備
            self._setup_session()
            
            return True
        except Exception as e:
            print(f"ログインエラー: {str(e)}")
            return False


    def get_products_from_page(self, url, page_number):
        """
        スーパーデリバリーの商品一覧ページから直接商品情報を取得します
        
        Args:
            url (str): 商品一覧ページのURL
            page_number (int): ページ番号（表示用）
        
        Returns:
            list: 商品データのリスト
        """
        products_data = []
        print(f"現在 {page_number} ページ目をスクレイピング中... ({url})")
        
        try:
            # Seleniumを使ってページを取得
            print("Seleniumでページの取得を開始...")
            self.browser.get(url)
            # 商品ボックスが表示されるまで待機（最大10秒）
            self.wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.plist-detail-box")))
            html_content = self.browser.page_source
            print("ページの取得完了")
            
            soup = BeautifulSoup(html_content, 'html.parser')
            
            # 卸業者名をページヘッダーから取得
            supplier_name = "不明"  # デフォルト値
            supplier_element = soup.select_one("h1.dl-name a.dl-name-txt")
            if supplier_element:
                supplier_name = supplier_element.text.strip()
                print(f"卸業者名: {supplier_name}")
            else:
                print("卸業者名の要素が見つかりません")
            
            # 各商品ボックスを探す
            print("商品ボックスを検索中...")
            product_boxes = soup.select("div.plist-detail-box")
            print(f"商品ボックスの数: {len(product_boxes)}")
            
            if not product_boxes:
                print(f"ページ {page_number} の商品リストが見つかりません")
                return products_data
            
            # 各商品ボックスからデータを抽出
            for box in product_boxes:
                try:
                    # 商品情報が入っているテーブルを取得
                    set_list_table = box.select_one("table.set-list")
                    
                    if not set_list_table:
                        continue
                    
                    # テーブル内の全ての行を取得（見出し行を除く）
                    rows = set_list_table.select("tbody > tr:not(:first-child)")
                    
                    # 3行ごとにグループ化処理
                    i = 0
                    while i < len(rows):
                        # エラー行や空の行をスキップ
                        if "errors_" in rows[i].get("id", "") or not rows[i].get("class"):
                            i += 1
                            continue
                        
                        # 商品情報を持つ行が3つ揃っているか確認
                        if i+2 < len(rows):
                            name_row = rows[i]      # 商品名とJANコード
                            price_row = rows[i+1]   # 価格
                            set_row = rows[i+2]     # セット数
                            
                            # 商品名を取得 (F12で確認した要素を使用)
                            name_cell = name_row.select_one("td.border-rt.co-p5.co-align-left")
                            if not name_cell:
                                i += 3
                                continue
                                
                            name_text = name_cell.get_text(strip=True)
                            product_name = name_text.split("JAN：")[0].strip() if "JAN：" in name_text else name_text
                            
                            # JANコードを取得
                            jan_element = name_cell.select_one("div.co-fcgray.co-fs12")
                            jan_code = ""
                            if jan_element:
                                jan_text = jan_element.get_text(strip=True)
                                jan_match = re.search(r'JAN：(\d+)', jan_text)
                                if jan_match:
                                    jan_code = jan_match.group(1)
                            
                            # 価格を取得
                            price_cell = price_row.select_one("td.border-r.co-align-right.co-pr5")
                            price = ""
                            if price_cell:
                                # まず割引価格（cmp-price）があるか確認
                                cmp_price_element = price_cell.select_one("p.cmp-price")
                                if cmp_price_element:
                                    price_text = cmp_price_element.get_text(strip=True)
                                    price_match = re.search(r'¥([\d,]+)', price_text)
                                    if price_match:
                                        price = price_match.group(1).replace(',', '')
                                
                                # 割引価格が見つからなければ通常価格を取得
                                if not price:
                                    # 通常の価格取得（list-priceまたは直接のテキスト）
                                    list_price_element = price_cell.select_one("p.list-price span")
                                    if list_price_element:
                                        price_text = list_price_element.get_text(strip=True)
                                    else:
                                        price_text = price_cell.get_text(strip=True)
                                    
                                    price_match = re.search(r'¥([\d,]+)', price_text)
                                    if price_match:
                                        price = price_match.group(1).replace(',', '')
                            
                            # セット数を取得
                            set_cell = set_row.select_one("td.set-num span.text-newline")
                            set_number = "1"  # デフォルト値
                            if set_cell:
                                set_text = set_cell.get_text(strip=True)
                                set_match = re.search(r'（(\d+)点）', set_text)
                                if set_match:
                                    set_number = set_match.group(1)
                            
                            # 商品データをリストに追加
                            if product_name and jan_code and price:
                                products_data.append([
                                    supplier_name,     # 卸業者名
                                    product_name,      # 商品名
                                    jan_code,          # JANコード
                                    price,             # 価格
                                    set_number         # セット数
                                ])
                            
                            # 次のグループへ
                            i += 3
                        else:
                            # 残りの行が3つ未満なら終了
                            break
                    
                except Exception as e:
                    print(f"商品データ取得エラー: {str(e)}")
                    continue
        
        except Exception as e:
            print(f"ページ {page_number} の取得エラー: {str(e)}")
        
        print(f"取得した商品データ数: {len(products_data)}")
        return products_data


    
    def scrape_all_targets(self):
        """
        設定ファイルに指定されたすべてのターゲットページをスクレイピングします
        
        Returns:
            int: 取得した商品データの総数
        """
        # 実行時間測定開始
        self.start_time = time.time()
        
        # CSVを初期化
        self.prepare_csv()
        
        # ログイン（設定ファイルの情報を使用）
        if not self.login():
            print("ログインに失敗しました")
            return 0
        
        # 各ターゲットページの処理
        total_items = 0
        
        for target in self.sudeli_config['target_pages']:
            print(f"\n===== {target['name']} の処理を開始 =====")
            
            # URLとページ範囲を取得
            base_url = target['url']
            sort = target.get('sort', '')  # ソート条件（指定がなければ空文字）
            start_page = target.get('start_page', 1)
            end_page = target.get('end_page', 1)
            
            # 各ページの処理
            for page in range(start_page, end_page + 1):
                # URLを構築（スーパーデリバリーの形式に合わせる）
                # ソートパラメータが指定されている場合のみURLに追加
                if sort:
                    page_url = f"{base_url}/all/{page}/?so={sort}&vi=3"
                else:
                    page_url = f"{base_url}/all/{page}/?vi=3"
                
                # 商品データを直接取得
                page_data = self.get_products_from_page(page_url, page)
                
                # データを保存
                if page_data:
                    self.save_to_csv(page_data)
                    total_items += len(page_data)
                    print(f"ページ {page} のデータ ({len(page_data)}件) をCSVに保存しました")
        
        # 実行時間測定終了
        self.end_time = time.time()
        elapsed_time = self.end_time - self.start_time
        
        print(f"\n===== スクレイピング完了 - 合計 {total_items} 件 =====")
        print(f"実行時間: {elapsed_time:.2f} 秒")
        
        return total_items

In [16]:
# スクレイパーのインスタンスを作成して実行するコード
if __name__ == "__main__":
    # スクレイパーのインスタンスを作成
    scraper = SudeliScraper(headless_mode=False)

    try:
        # 設定ファイルに基づいて全ターゲットをスクレイピング
        scraper.scrape_all_targets()
    finally:
        # 終了処理
        scraper.close()

2025-03-21 16:16:29,849 - INFO - ログ機能の初期化が完了しました: C:\Users\inato\Documents\amazon-research\logs\sudeli_20250321_161629.log
2025-03-21 16:16:29,849 - INFO - ====== WebDriver manager ======


設定ファイルパス: C:\Users\inato\Documents\amazon-research\config\settings.yaml
スーパーデリバリーのログイン情報を環境変数から設定しました
設定ファイルを正常に読み込みました
ログファイル出力先: C:\Users\inato\Documents\amazon-research\logs\sudeli_20250321_161629.log
CSVファイル出力先: C:\Users\inato\Documents\amazon-research\data\sudeli_scraping.csv


2025-03-21 16:16:31,234 - INFO - Get LATEST chromedriver version for google-chrome
2025-03-21 16:16:31,288 - INFO - Get LATEST chromedriver version for google-chrome
2025-03-21 16:16:31,307 - INFO - Driver [C:\Users\inato\.wdm\drivers\chromedriver\win64\134.0.6998.90\chromedriver-win32/chromedriver.exe] found in cache


ブラウザの初期化に成功しました
CSVファイル C:\Users\inato\Documents\amazon-research\data\sudeli_scraping.csv を初期化しました

===== 健康フーズ の処理を開始 =====
現在 1 ページ目をスクレイピング中... (https://www.superdelivery.com/p/do/dpsl/1000090/all/1/?so=newly&vi=3)
Seleniumでページの取得を開始...
ページの取得完了
卸業者名: 健康フーズ
商品ボックスを検索中...
商品ボックスの数: 20
取得した商品データ数: 39
ページ 1 のデータ (39件) をCSVに保存しました
現在 2 ページ目をスクレイピング中... (https://www.superdelivery.com/p/do/dpsl/1000090/all/2/?so=newly&vi=3)
Seleniumでページの取得を開始...
ページの取得完了
卸業者名: 健康フーズ
商品ボックスを検索中...
商品ボックスの数: 20
取得した商品データ数: 36
ページ 2 のデータ (36件) をCSVに保存しました
現在 3 ページ目をスクレイピング中... (https://www.superdelivery.com/p/do/dpsl/1000090/all/3/?so=newly&vi=3)
Seleniumでページの取得を開始...
ページの取得完了
卸業者名: 健康フーズ
商品ボックスを検索中...
商品ボックスの数: 20
取得した商品データ数: 40
ページ 3 のデータ (40件) をCSVに保存しました
現在 4 ページ目をスクレイピング中... (https://www.superdelivery.com/p/do/dpsl/1000090/all/4/?so=newly&vi=3)
Seleniumでページの取得を開始...
ページの取得完了
卸業者名: 健康フーズ
商品ボックスを検索中...
商品ボックスの数: 20
取得した商品データ数: 40
ページ 4 のデータ (40件) をCSVに保存しました
現在 5 ページ目をスクレイピング中... (https://www.superdeliver